In [1]:
# Imports

import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import os,sys,math,itertools

import torch
import pandas as pd

from scipy import linalg as la

In [2]:
class GRAPH():
    def __init__(self,N,f=lambda x: x,energy=lambda x: np.linalg.norm(x),MAXHIST=10,norm_energy=1):
        # Parameters
        self.N = N
        # Weight matrix, bias and non-linearity
        self.W = rnd.randn(N,N)
        self.b = rnd.randn(N)
        self.f = f
        self.energy = energy
        # Init random state
        self.norm_energy = norm_energy
        self.s = rnd.randn(N)
        self.s = self.__normalize(self.s)
        self.e = self.energy(self.s)
        # State and energy history
        self.MAXHIST = MAXHIST
        self.S = []
        self.E = []
        self.S.append(self.s)
        self.E.append(self.e)
        # Return
        return
    def set_state(self,s):
        self.s = s
        self.s = self.__normalize(self.s)
        self.e = self.energy(self.s)
        self.__update_states_hist(s)
        self.__update_energy_hist(self.e)
        return
    def update(self,steps=1):
        for step in range(steps):
            self.s = self.f(np.einsum('ij,j->i',self.W,self.s) + self.b)
            self.s = self.__normalize(self.s)
            self.e = self.energy(self.s)
            self.__update_states_hist(self.s)
            self.__update_energy_hist(self.e)
        return
    def __normalize(self,x):
        y = x
        if self.norm_energy>0:
            y /= self.energy(y)
            y *= self.norm_energy
        return y
    def __update_states_hist(self,x):
        self.S.append(x)
        if self.MAXHIST>0:
            if len(self.S)>self.MAXHIST: self.S = self.S[-self.MAXHIST:]
        return
    def __update_energy_hist(self,x):
        self.E.append(x)
        if self.MAXHIST>0:
            if len(self.E)>self.MAXHIST: self.E = self.E[-self.MAXHIST:]
        return

In [3]:
def generate_correlated_rnd(dim,R=np.array([[1,0],[0,1]])):
    assert R.shape[0]==R.shape[1],'Input R not a proper correlation matrix'
    L = np.linalg.cholesky(R)
    X = L@rnd.randn(R.shape[0],dim)
    return X

# Investigating stable energy states of the graph

In [13]:
# Network setup

relu = lambda x: (x*(x>0)).astype(np.float32)
G =GRAPH(N=25,MAXHIST=250,f=relu)

In [14]:
G.update(250)

In [18]:
import pygame,sys
from pygame.time import delay
from pygame.locals import *

In [16]:
COLORMAP = (255*np.array(G.S[1:]).reshape(len(G.S)-1,5,5)).astype(int)

In [20]:
pygame.init()

DISPLAY=pygame.display.set_mode((500,500),0,32)
DISPLAY.fill((255,255,255))

ctr = 0
while True:
    for event in pygame.event.get():
        if event.type==QUIT:
            pygame.quit()
            sys.exit()
    for nrow in range(5):
        for ncol in range(5):
            pygame.draw.rect(DISPLAY,(0,0,COLORMAP[min(ctr,len(G.S)-1)-1,nrow,ncol]),(100*ncol,100*nrow,100,100))
    ctr += 1
    pygame.display.update()
    delay(1000)

SystemExit: 

In [14]:
COLORMAP = (255*rnd.randint(0,255,size=(5,5,2))

In [12]:
len(G.S)

101

In [10]:
COLORMAP[1,:,:]

array([[136,  45,  36,  41,   0],
       [  0,  68,   0,   0, 104],
       [ 26,   0,   0,   0,   0],
       [  0,  30,   0,   0, 116],
       [ 58,  24,  38,   8,  69]])

In [11]:
COLORMAP[99,:,:]

array([[105,  26,  25,   0,  14],
       [  0,  77,   0,   0, 123],
       [  3,   0,  57,   0,   0],
       [  0,   0,   0,   6,  68],
       [ 38, 126,   0,   0,  74]])